# West Virginia Production dataset


In [1]:
# python/pandas preamble
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import zipfile, os

# different sets contain input files that are formatted similarly
set1_dir = './sources/set1/'
set2_dir = './sources/set2/'
set3_dir = './sources/set3/'
set4_dir = './sources/set4/'
datadir = './sources/'
outdir = './out/'

# Get list of Fracked wells 

## ...from FracFocus
Less than ideal to get a list of fracked wells from FracFocus.  It will have to do for now...


In [ ]:
pkl = 'c:/MyDocs/sandbox/data/O&G/fracfocus/out/currentData/well.pkl'
well = pd.read_pickle(pkl)
well.info()
ffwvapi = list(well[well.StateNumber==47].API10.unique())

## ... from WV production

### 2018
This is supposed to have only H6A Horizontal wells. So all API are valid.

In [ ]:
wvapi = []
klst = ['API '] 
flst = os.listdir(set4_dir)
for zfn in flst:
    with zipfile.ZipFile(set4_dir+zfn) as z:
        for fn in z.namelist():
            with z.open(fn) as f:
                print(f' -- processing {fn}')
                t = pd.read_excel(f,usecols=klst) # read the file
                #print(t.columns)
                t.columns = ['API']
                t.API = t.API.astype('str').str[0:10]
            for a in list(t.API):
                wvapi.append(a)
                
len(set(wvapi))

### 2016-2017
Use the API that are flagged as Horizontal of some kind.

In [ ]:
klst = ['API','Horizontal Well Indicator ']
flst = os.listdir(set2_dir)
dlst = []
for zfn in flst:
    with zipfile.ZipFile(set2_dir+zfn) as z:
        for fn in z.namelist():
            with z.open(fn) as f:
                print(f' -- processing {fn}')
                t = pd.read_excel(f,skiprows=3,usecols=klst) # read the file
                t.columns = ['API','Hwell']
                t.API = t.API.astype('str').str[0:10]
                t = t[t.Hwell.str[0]== 'H']
                print(len(t))
            for a in list(t.API):
                wvapi.append(a)

wvapi = list(set(wvapi))
print(f'Len of Horiz. well list: {len(wvapi)}')

# Metadata from WVGES

See website <http://www.wvgs.wvnet.edu/www/datastat/devshales.htm>  for lots of resources.  This includes
an excel sheet with lots of metadata on each unconventional well. We use that here to filter out conventional wells and for lat/lon, etc.

**Note that the original file is in read-only lock.  You must make an explicit copy within Excel to successfully read in these data.**

In [2]:
meta = pd.read_excel(datadir+'Copy of WVGES Marcellus Wells.xlsx',sheet_name='WV Marcellus Prod',
                    skiprows=1)
# meta = pd.read_excel(datadir+'WellLocation(08-22-2016).xlsx',engine='xlrd',sheet_name=0)

In [3]:
meta.head()
meta.API = meta.API.astype('str').str[:10]
wvapi = list(meta.API.unique())
# wvapi

# Set 1 - 2007-2011 and 2013-2015

In [4]:
# def change_name(row,yr):
#     xlate = {'GAS_PRODUCTION_':'Gas_','OIL_PRODUCTION_':'Oil_','LNG_PROD_':'LNG_','NGL_PROD_':'LNG_'}
#     mon = {'JANUARY':'01', 'FEBRUARY':'02', 'MARCH':'03', 'APRIL':'04', 'MAY':'05', 'JUNE':'06',
#            'JULY':'07', 'AUGUST':'08', 'SEPTEMBER':'09', 'OCTOBER':'10', 'NOVEMBER':'11', 'DECEMBER':'12',
#            'JAN':'01', 'FEB':'02', 'MAR':'03', 'APR':'04', 'MAY':'05', 'JUN':'06',
#            'JUL':'07', 'AUG':'08', 'SEP':'09', 'OCT':'10', 'NOV':'11', 'DEC':'12'}

klst = ['YEAR', 'API', 'GAS_PRODUCTION_JANUARY',
       'GAS_PRODUCTION_FEBRUARY', 'GAS_PRODUCTION_MARCH',
       'GAS_PRODUCTION_APRIL', 'GAS_PRODUCTION_MAY', 'GAS_PRODUCTION_JUNE',
       'GAS_PRODUCTION_JULY', 'GAS_PRODUCTION_AUGUST',
       'GAS_PRODUCTION_SEPTEMBER', 'GAS_PRODUCTION_OCTOBER',
       'GAS_PRODUCTION_NOVEMBER', 'GAS_PRODUCTION_DECEMBER',
       'OIL_PRODUCTION_JANUARY',
       'OIL_PRODUCTION_FEBRUARY', 'OIL_PRODUCTION_MARCH',
       'OIL_PRODUCTION_APRIL', 'OIL_PRODUCTION_MAY', 'OIL_PRODUCTION_JUNE',
       'OIL_PRODUCTION_JULY', 'OIL_PRODUCTION_AUGUST',
       'OIL_PRODUCTION_SEPTEMBER', 'OIL_PRODUCTION_OCTOBER',
       'OIL_PRODUCTION_NOVEMBER', 'OIL_PRODUCTION_DECEMBER']
#klst = [1,2,3]
flst = os.listdir(set1_dir)
dlst = []
for zfn in flst:
    with zipfile.ZipFile(set1_dir+zfn) as z:
        for fn in z.namelist():
            with z.open(fn) as f:
                print(f' -- processing {fn}')
                t = pd.read_excel(f,usecols=klst) # read the file
                #print(t.columns)
                yr = int(t.YEAR.mean())
                t.columns = ['Year','API',f'Gas_{yr}M01',f'Gas_{yr}M02',f'Gas_{yr}M03',
                            f'Gas_{yr}M04',f'Gas_{yr}M05',f'Gas_{yr}M06',f'Gas_{yr}M07',f'Gas_{yr}M08',
                            f'Gas_{yr}M09',f'Gas_{yr}M10',f'Gas_{yr}M11',f'Gas_{yr}M12',
                            f'Oil_{yr}M01',f'Oil_{yr}M02',f'Oil_{yr}M03',
                            f'Oil_{yr}M04',f'Oil_{yr}M05',f'Oil_{yr}M06',f'Oil_{yr}M07',f'Oil_{yr}M08',
                            f'Oil_{yr}M09',f'Oil_{yr}M10',f'Oil_{yr}M11',f'Oil_{yr}M12']
                t.API = t.API+4700000000
                t.API = t.API.astype('str').str[0:10]
                t = t[t.API.isin(wvapi)]
                t = t.drop(['Year'],axis=1)
                m = pd.melt(t,id_vars='API')
                dlst.append(m)

set1_df = pd.concat(dlst,sort=True)



 -- processing 2007Production.xls
 -- processing 2008Production.xls
 -- processing 2009Production.xls
 -- processing 2010Production.xls
 -- processing 2011Production.xls
 -- processing 2013 production 08-25-2016.xlsx
 -- processing 2014 WV Production Data Update 04-29-2016.xlsx
 -- processing 2015 West Virginia Production 08-23-2016.xlsx


In [5]:
dup = pd.concat(g for _, g in set1_df.groupby(["API","variable"]) if len(g) > 1)
dup.API.unique()
# quite a few duplicates

array(['4700502175', '4700702494', '4701705592', '4702300023',
       '4703305448', '4703305590', '4703502725', '4703502763',
       '4703905967', '4704105231', '4704303377', '4704303378',
       '4704303379', '4704303380', '4704303381', '4704303382',
       '4704303387', '4704303388', '4704303390', '4704303391',
       '4704303405', '4704303407', '4704502024', '4704502037',
       '4704502038', '4704502104', '4704502119', '4704502123',
       '4704502135', '4704502137', '4704901900', '4704902038',
       '4705101093', '4705300437', '4705901202', '4705901804',
       '4705901805', '4705901836', '4705901863', '4705901866',
       '4705901931', '4705901947', '4705901952', '4705901957',
       '4707700173', '4707901476', '4708301143', '4708301211',
       '4708506359', '4708509333', '4708509571', '4708509990',
       '4708704509', '4708704518', '4708704582', '4708704601',
       '4708704602', '4709101210', '4709101220', '4709101290',
       '4709703503', '4709703515', '4709703516', '47097

In [6]:
set1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357504 entries, 0 to 73175
Data columns (total 3 columns):
API         357504 non-null object
value       357504 non-null float64
variable    357504 non-null object
dtypes: float64(1), object(2)
memory usage: 10.9+ MB


# Set 2: 2016-17

In [5]:
klst = ['Year ', 'API',    # in the 2016 file, there's an extra space at the end of five col names 
        'Gas_January',
       'Gas_Feburary ', 'Gas_March ',  # February is misspelled
       'Gas_April', 'Gas_May', 'Gas_June',
       'Gas_July', 'Gas_August',
       'Gas_September', 'Gas_October',
       'Gas_November', 'Gas_December',
       'Oil_January',
       'Oil_Feburary ', 'Oil_March ',
       'Oil_April', 'Oil_May', 'Oil_June',
       'Oil_July', 'Oil_August',
       'Oil_September', 'Oil_October',
       'Oil_November', 'Oil_December']
#klst = [1,2,3]
flst = os.listdir(set2_dir)
dlst = []
for zfn in flst:
    with zipfile.ZipFile(set2_dir+zfn) as z:
        for fn in z.namelist():
            with z.open(fn) as f:
                print(f' -- processing {fn}')
                t = pd.read_excel(f,skiprows=3,usecols=klst) # read the file
                #print(t.columns)
                yr = int(t['Year '].mean())
                t.columns = ['Year','API',f'Gas_{yr}M01',f'Gas_{yr}M02',f'Gas_{yr}M03',
                            f'Gas_{yr}M04',f'Gas_{yr}M05',f'Gas_{yr}M06',f'Gas_{yr}M07',f'Gas_{yr}M08',
                            f'Gas_{yr}M09',f'Gas_{yr}M10',f'Gas_{yr}M11',f'Gas_{yr}M12',
                            f'Oil_{yr}M01',f'Oil_{yr}M02',f'Oil_{yr}M03',
                            f'Oil_{yr}M04',f'Oil_{yr}M05',f'Oil_{yr}M06',f'Oil_{yr}M07',f'Oil_{yr}M08',
                            f'Oil_{yr}M09',f'Oil_{yr}M10',f'Oil_{yr}M11',f'Oil_{yr}M12']
                t.API = t.API.astype('str').str[0:10]
                t = t[t.API.isin(wvapi)]
                t = t.drop(['Year'],axis=1)
                m = pd.melt(t,id_vars='API')
                dlst.append(m)

set2_df = pd.concat(dlst,sort=True)


 -- processing 2016 West Virginia Production Data 11-13-2017.xlsx
 -- processing 2017 West Virginia Production Totals 06-06-2018.xlsx


In [15]:
# dup = pd.concat(g for _, g in set2_df.groupby(["API","variable"]) if len(g) > 1)
# dup.API.unique()

# Set 3 - 2012 (an odd file)

In [6]:
klst = ['PRODUCTION_YEAR', 'WELL_API', 'GAS_PRODUCTION_JANUARY',
       'GAS_PRODUCTION_FEBRUARY', 'GAS_PRODUCTION_MARCH',
       'GAS_PRODUCTION_APRIL', 'GAS_PRODUCTION_MAY', 'GAS_PRODUCTION_JUNE',
       'GAS_PRODUCTION_JULY', 'GAS_PRODUCTION_AUGUST',
       'GAS_PRODUCTION_SEPTEMBER', 'GAS_PRODUCTION_OCTOBER',
       'GAS_PRODUCTION_NOVEMBER', 'GAS_PRODUCTION_DECEMBER',
       'OIL_PRODUCTION_JANUARY',
       'OIL_PRODUCTION_FEBRUARY', 'OIL_PRODUCTION_MARCH',
       'OIL_PRODUCTION_APRIL', 'OIL_PRODUCTION_MAY', 'OIL_PRODUCTION_JUNE',
       'OIL_PRODUCTION_JULY', 'OIL_PRODUCTION_AUGUST',
       'OIL_PRODUCTION_SEPTEMBER', 'OIL_PRODUCTION_OCTOBER',
       'OIL_PRODUCTION_NOVEMBER', 'OIL_PRODUCTION_DECEMBER']
#klst = [1,2,3]
flst = os.listdir(set3_dir)
dlst = []
for zfn in flst:
    with zipfile.ZipFile(set3_dir+zfn) as z:
        for fn in z.namelist():
            with z.open(fn) as f:
                print(f' -- processing {fn}')
                t = pd.read_excel(f,usecols=klst) # read the file
                #print(t.columns)
                yr = int(t.PRODUCTION_YEAR.mean())
                t.columns = ['Year','API',f'Gas_{yr}M01',f'Gas_{yr}M02',f'Gas_{yr}M03',
                            f'Gas_{yr}M04',f'Gas_{yr}M05',f'Gas_{yr}M06',f'Gas_{yr}M07',f'Gas_{yr}M08',
                            f'Gas_{yr}M09',f'Gas_{yr}M10',f'Gas_{yr}M11',f'Gas_{yr}M12',
                            f'Oil_{yr}M01',f'Oil_{yr}M02',f'Oil_{yr}M03',
                            f'Oil_{yr}M04',f'Oil_{yr}M05',f'Oil_{yr}M06',f'Oil_{yr}M07',f'Oil_{yr}M08',
                            f'Oil_{yr}M09',f'Oil_{yr}M10',f'Oil_{yr}M11',f'Oil_{yr}M12']
                t.API = t.API.str.replace('-','')  # drop the dashes; already a 10 digit string
                t = t[t.API.isin(wvapi)]
                t = t.drop(['Year'],axis=1)
                m = pd.melt(t,id_vars='API')
                dlst.append(m)

set3_df = pd.concat(dlst,sort=True)



 -- processing Production2012.xlsx


In [17]:
dup = pd.concat(g for _, g in set3_df.groupby(["API","variable"]) if len(g) > 1)
dup.API.unique()

array(['4701304145', '4701304623', '4701304624', '4701705402',
       '4703305448', '4704502024', '4704502104', '4704502119',
       '4708506359', '4708509348', '4708704601', '4708704602'],
      dtype=object)

# Set 4 - 2018

In [7]:
klst = ['Year', 'API ', 
        'Jan_Gas', 'Feb_Gas', 'Mar_Gas', 'Apr_Gas', 'May_Gas', 'Jun_Gas',
        'Jul_Gas', 'Aug_Gas', 'Sep_Gas', 'Oct_Gas', 'Nov_Gas', 'Dec_Gas',
        'Jan_Oil', 'Feb_Oil', 'Mar_Oil', 'Apr_Oil', 'May_Oil', 'Jun_Oil',
        'Jul_Oil', 'Aug_Oil', 'Sep_Oil', 'Oct_Oil', 'Nov_Oil', 'Dec_Oil']
flst = os.listdir(set4_dir)
dlst = []
for zfn in flst:
    with zipfile.ZipFile(set4_dir+zfn) as z:
        for fn in z.namelist():
            with z.open(fn) as f:
                print(f' -- processing {fn}')
                t = pd.read_excel(f,usecols=klst) # read the file
                #print(t.columns)
                yr = int(t['Year'].mean())
                t.columns = ['Year','API',f'Gas_{yr}M01',f'Gas_{yr}M02',f'Gas_{yr}M03',
                            f'Gas_{yr}M04',f'Gas_{yr}M05',f'Gas_{yr}M06',f'Gas_{yr}M07',f'Gas_{yr}M08',
                            f'Gas_{yr}M09',f'Gas_{yr}M10',f'Gas_{yr}M11',f'Gas_{yr}M12',
                            f'Oil_{yr}M01',f'Oil_{yr}M02',f'Oil_{yr}M03',
                            f'Oil_{yr}M04',f'Oil_{yr}M05',f'Oil_{yr}M06',f'Oil_{yr}M07',f'Oil_{yr}M08',
                            f'Oil_{yr}M09',f'Oil_{yr}M10',f'Oil_{yr}M11',f'Oil_{yr}M12']
                t.API = t.API.astype('str').str[0:10]
                t = t[t.API.isin(wvapi)]
                t = t.drop(['Year'],axis=1)
                m = pd.melt(t,id_vars='API')
                dlst.append(m)

set4_df = pd.concat(dlst,sort=True)


 -- processing 2018 Q3 WV Production Horizontal H6A Wells .xlsx


In [19]:
# dup = pd.concat(g for _, g in set4_df.groupby(["API","variable"]) if len(g) > 1)
# dup.API.unique()

In [8]:
whole = pd.concat([set1_df,set2_df,set3_df,set4_df,],sort=True)
set1_df.head()
# in the years before 2016, there are occasionally wells with duplicate records, often with one
# of them having '0' as quantity, so we will take the max of the duplcates.
whole = whole.groupby(['API','variable'],as_index=False).max()
print(f'Records = {len(whole)} and {len(whole.API.unique())} unique wells')
whole.info()
whole.to_pickle(outdir+'ver1.pkl')

Records = 602232 and 3684 unique wells
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602232 entries, 0 to 602231
Data columns (total 3 columns):
API         602232 non-null object
variable    602232 non-null object
value       537738 non-null float64
dtypes: float64(1), object(2)
memory usage: 13.8+ MB


## Compare these numbers to those in the WVGES dataset

In [12]:
whole['typ1'] = whole.variable.str[:3]
whole['year1'] = whole.variable.str[4:8].astype('int64')
whole['month1'] = whole.variable.str[-2:].astype('int64')
whole.head()

,API,variable,value,typ1,year1,month1
0,4700102885,Gas_2008M01,0.0,Gas,2008,1
1,4700102885,Gas_2008M02,0.0,Gas,2008,2
2,4700102885,Gas_2008M03,956.0,Gas,2008,3
3,4700102885,Gas_2008M04,1082.0,Gas,2008,4
4,4700102885,Gas_2008M05,1397.0,Gas,2008,5


In [22]:
whole.columns = ['API', 'variable1', 'value1', 'typ', 'YEAR', 'month']
whole.typ = whole.typ.str.upper()
whole.head()

,API,variable1,value1,typ,YEAR,month
0,4700102885,Gas_2008M01,0.0,GAS,2008,1
1,4700102885,Gas_2008M02,0.0,GAS,2008,2
2,4700102885,Gas_2008M03,956.0,GAS,2008,3
3,4700102885,Gas_2008M04,1082.0,GAS,2008,4
4,4700102885,Gas_2008M05,1397.0,GAS,2008,5


In [28]:
ver2 = pd.read_pickle(outdir+'ver2.pkl')
ver2['month'] = ver2.yrmo.str[-2:].astype('int64')
ver2.API = ver2.API.astype('str').str[:10]
ver2.head()
ver2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 855492 entries, 0 to 855491
Data columns (total 7 columns):
API         855492 non-null object
YEAR        855492 non-null int64
variable    855492 non-null object
value       632650 non-null float64
typ         855492 non-null object
yrmo        855492 non-null object
month       855492 non-null int64
dtypes: float64(1), int64(2), object(4)
memory usage: 52.2+ MB


In [29]:
mg = pd.merge(whole,ver2,on=['API','YEAR','month','typ'])
mg.head()

,API,variable1,value1,typ,YEAR,month,variable,value,yrmo
0,4700102885,Gas_2008M01,0.0,GAS,2008,1,G_JAN,0.0,Y2008M01
1,4700102885,Gas_2008M02,0.0,GAS,2008,2,G_FEB,0.0,Y2008M02
2,4700102885,Gas_2008M03,956.0,GAS,2008,3,G_MAR,956.0,Y2008M03
3,4700102885,Gas_2008M04,1082.0,GAS,2008,4,G_APR,1082.0,Y2008M04
4,4700102885,Gas_2008M05,1397.0,GAS,2008,5,G_MAY,1397.0,Y2008M05


In [32]:
test = mg[mg.typ=='GAS']
res = test.value - test.value1
print(f'Total rec: {len(res)}, number non-matches: {len(res[res>0])}')

Total rec: 275935, number non-matches: 265


In [21]:
out = whole.pivot(index='API',columns='variable')
out.columns = out.columns.get_level_values(1)
out.head()
out.to_csv(outdir+'temp.csv')

In [ ]:
locat = pd.read_excel(datadir+'WellLocation(08-22-2016).xlsx')

In [ ]:
locat.info()

